In [1]:
import pandas as pd
import numpy as np
import requests
from bs4 import BeautifulSoup

In [2]:
import requests
from bs4 import BeautifulSoup
from urllib.parse import urljoin, quote
import re

def parse_children_page(url):
    response = requests.get(url)
    
    # Перевірка на успішність відповіді
    if response.status_code == 200:
        # Парсимо HTML за допомогою BeautifulSoup
        soup = BeautifulSoup(response.text, "html.parser")
        
        # Знаходимо елементи, які містять інформацію (з актуальним класом)
        code = soup.find_all(class_="container-fluid passport")  # Актуалізуйте клас, якщо він інший

        # Перебір знайдених елементів у code
        for element in code:
            # Знаходимо ім'я
            name = element.find(class_="passport__title-name")
            if name:
                print(name.get_text(strip=True))
            else:
                print("Ім'я не знайдено в цьому елементі.")                
            # passport__title-number
            #id
            id = element.find(class_="passport__title-number").get_text(strip=True)
            number = re.search(r'\d+', id).group()
            
            print("id дитини:",number)
            
            # Виводимо фото
            photo = element.find(class_="js-lightBoxGallery").find('img').get('src')
            if photo:
                print("photo src=",photo)
            else:
                print("Фото не знайдено в цьому елементі.")

            #region
            region = element.find("ul").find_all("li")[1]
            if region:
                print("Region:",region.find_all("span")[1].get_text(strip=True))
            else:
                print("region not fount")
            
            #siblings
            siblings = element.find("ul").find_all("li")[2]
            if siblings:
                print("Siblings:",siblings.find_all("span")[1].get_text(strip=True))
            else:
                print("siblings not fount")

            #отримати кількість
            if (siblings.find_all("span")[1].get_text(strip=True) == "є"):
                siblingsCount = element.find(class_='passport__content-inner--brother').find_all("li")
                print("siblings count",len(siblingsCount))

            #дістати інформацію про братів і сестер
            if (siblings.find_all("span")[1].get_text(strip=True) == "є"):
                siblingsArr = element.find(class_='passport__content-inner--brother').find_all("li")
                for sibling in siblingsArr:
                    childLink = sibling.find('a').get('href')
                    fullLink = "https://www.msp.gov.ua"+childLink
                    print(fullLink)
                    #відкрити посилання і дістати інформацію
                    responseSibl = requests.get(fullLink)
                    soupSibl = BeautifulSoup(responseSibl.text, "html.parser")
                    codeSibl = soupSibl.find_all(class_="container-fluid passport")
                    for childSib in codeSibl:
                        nameSibl = childSib.find(class_="passport__title-name")
                        if nameSibl:
                            print(nameSibl.get_text(strip=True))
                        else:
                            print("Ім'я не знайдено в цьому елементі.")  
                        

            
            
            #invalid
            invalid = element.find("ul").find_all("li")[3]
            if invalid:
                print("Invalid:",invalid.find_all("span")[1].get_text(strip=True))
            else:
                print("ivnalid not fount")
            
            #form
            form = element.find("ul").find_all("li")[4]
            if form:
                print("Form:",form.find_all("span")[1].get_text(strip=True))
            else:
                print("form not fount")
            
            #contacts
            contacts = element.find("ul").find_all("li")[5]
            if contacts:
                print("Contacts:",contacts.find_all("span")[1].get_text(strip=True))
            else:
                print("contacts not fount")

            print("##################################################")
            
            # Виводимо список братів і сестер
            #siblings = element.find(class_="passport__content-inner--brother").find_all("li")
            #for sibling in siblings:
            #    print(sibling)
            #print(f"Всього рідних братів: {len(siblings)}")

            # Перехід до наступного блоку
            next_step = element.find(class_="btn btn-default passport__footer-next")
            next_step = 0 #затичка
            if next_step:
                onclick_text = next_step.get("onclick")
                
                # Витягуємо відносний URL із onclick
                start_index = onclick_text.find("'") + 1
                end_index = onclick_text.rfind("'")
                relative_url = onclick_text[start_index:end_index]
                
                # Об'єднуємо з базовим URL
                base_url = "https://www.msp.gov.ua/children/"
                full_url = urljoin(base_url, relative_url)
                
                # Кодуємо для коректного URL
                encoded_url = quote(full_url, safe=":/?=&")
                parse_children_page(encoded_url)
                print("--------------------------------------")
                print("--------------------------------------")
            else:
                print("Кнопка переходу не знайдена")
    else:
        print("Не вдалося завантажити сторінку")

# Виклик функції з потрібним URL
#parse_children_page("https://www.msp.gov.ua/children/child.php?n=405181&form=*&male=*&age_from=3&age_to=3&region=*&brothers=*&needs=*")
parse_children_page("https://www.msp.gov.ua/children/child.php?n=405229&form=*&male=*&age_from=*&age_to=*&region=*&brothers=*&needs=*")


Юлія, 7 років
id дитини: 405229
photo src= ../files/children/05_children12.08.2024.zip/36630979.jpeg
Region: Донецька обл.
Siblings: є
siblings count 2
https://www.msp.gov.ua/children/child.php?n=405231
Ганна, 9 років
https://www.msp.gov.ua/children/child.php?n=405233
Андрій, 5 років
Invalid: немає
Form: національне усиновлення; опіка/піклування; прийомна сім'я та дитячий будинок сімейного типу;
Contacts: ССД Донецької ОДА: (06264) 8-52-23
##################################################
Кнопка переходу не знайдена


In [10]:
import pymysql
connection = pymysql.connect(
    host="localhost",
    user="root",
    password="root",
    database="test"
)

# Виконуємо запит
try:
    with connection.cursor() as cursor:
        query = "SELECT * FROM persons LIMIT 5"
        cursor.execute(query)
        result = cursor.fetchall()
        for row in result:
            print(row)
finally:
    connection.close()

In [ ]:
"CREATE TABLE children (id INT PRIMARY KEY AUTO_INCREMENT,name VARCHAR(100) NOT NULL,age INT,status VARCHAR(50),disability BOOLEAN DEFAULT 0,has_siblings BOOLEAN DEFAULT 0,siblings_links TEXT);"


In [11]:
import pymysql
connection = pymysql.connect(
    host="localhost",
    user="root",
    password="root",
    database="test"
)

# Виконуємо запит
try:
    with connection.cursor() as cursor:
        query = "CREATE TABLE children (id INT PRIMARY KEY AUTO_INCREMENT,name VARCHAR(100) NOT NULL,age INT,status VARCHAR(50),disability BOOLEAN DEFAULT 0,has_siblings BOOLEAN DEFAULT 0,siblings_links TEXT)"
        cursor.execute(query)
        result = cursor.fetchall()
        for row in result:
            print(row)
finally:
    connection.close()